Last post we looked at how to gain the inner boundary and center point of the carotid artery using contours and moments and In this post we will look at how to extract the outer boundary. First we load up all the code from the last post

In [17]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math
%matplotlib inline

I = cv2.imread("carotid.jpg")
carotid_img = cv2.imread("Carotid.jpg" )
copy = cv2.imread("Carotid.jpg" )

#Create the ranges of the mask
HSV = cv2.cvtColor(carotid_img, cv2.COLOR_BGR2HSV)
#Create a lower and upper range
#Create a lower and upper range
lower = np.array([0,141,39])
upper = np.array([87,255,255])
#make mask
mask_img = cv2.inRange(HSV, lower, upper)
img,contours, hierachery = cv2.findContours(mask_img,mode=cv2.RETR_EXTERNAL,method=cv2.CHAIN_APPROX_NONE)
carotid_img = cv2.drawContours(carotid_img, contours, contourIdx=-1, color=(0,255,255), thickness=1)
c = max(contours, key = cv2.contourArea)
c_img = cv2.drawContours(carotid_img, c, contourIdx=-1, color=(0,255,255), thickness=2)
moments = cv2.moments(c) 
cx = int(moments['m10']/moments['m00']) 
cy = int(moments['m01']/moments['m00']) 
cv2.circle(c_img, (cx, cy), 3, (255, 255, 255), -1) 

cv2.imwrite("contour.jpg",c_img)

True

![contour](contour.jpg)

We get a new mask using these configurations on HSV

In [ ]:
#we have the center point now mak a mask that only gets the external in
HSVlower = np.array([0,0,14])
HSVupper = np.array([179,35,255])


#define a mask of the artery
ExternalMask = cv2.inRange(HSV , HSVlower , HSVupper)
pointsOnBoundary =[]
cv2.imwrite("externalmask.jpg",ExternalMask)

![externalmask](externalmask.jpg)

We use the center point to produce radial lines using this algorithm. With help of this stack overflow answer:http://answers.opencv.org/question/56095/draw-lines-from-centroid-of-contour-at-given-angle-till-edge-of-contour/

In [18]:
angle =1;
length = 1
while angle<=360:
    #Following 2 lines adapted from
    p2x = int (cx + length * math.cos(angle * 3.14 / 180.0))
    p2yy = int (cy + length * math.sin(angle * 3.14 / 180.0))

    #if NOT its a black pxel
    if ExternalMask[p2yy , p2x] > 0:
        pt2 = (p2x, p2yy)
        cv2.line(img=I, pt1=(cx, cy), pt2=pt2, color=(255,255,0), thickness=1)
        cv2.circle(copy,(p2x, p2yy), 1,(0, 255, 255),-1)
        angle = angle + 2
        length = 2
        pointsOnBoundary.append(pt2)
    else:
        length = length + 2
cv2.imwrite("radial2.jpg",I)
cv2.imwrite("circcle.jpg",copy)

True

![radiallines](radial2.jpg)

![circle](circcle.jpg)

Looks good, now we draw the lines.

In [20]:
index =0
size = pointsOnBoundary.__len__()
while index < size -1 :
    this = pointsOnBoundary[index]
    next = pointsOnBoundary[index+1]
    cv2.line(img=copy , pt1= this , pt2=next , color=(0,255,0) , thickness = 1 )
    index = index + 1
cv2.imwrite("final.jpg",copy)

True

![final](final.jpg)

<h1> Testing </h1>
Now to test the algorithm on different carotid arteries, here are some results.

![test1](test1.jpg)

![test2](test2.jpg)

REFERENCES
http://answers.opencv.org/question/56095/draw-lines-from-centroid-of-contour-at-given-angle-till-edge-of-contour/